# Classification on CIFAR-10 dataset

In [69]:
# dependencies
import keras
from keras.datasets import cifar10
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from sklearn import metrics
import xgboost as xgb
from sklearn.model_selection import cross_val_score   

import time
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from matplotlib import pyplot as plt
import seaborn as sns

In [52]:
import warnings
warnings.filterwarnings("ignore")   

In [53]:
# define num_class
num_classes = 10

# load dataset keras will download cifar-10 datset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()  

In [54]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [55]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# (Optional)Convert class vectors to binary class matrices.
# y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [56]:
classesName = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## XGB for CIFAR 10 

In [57]:
baseDir =os.path.dirname(os.path.abspath('__file__')) + '/'

In [58]:
baseDir

'C:\\Users\\aish0\\OneDrive\\Documents\\College_study\\SML\\Final project/'

In [59]:
# Converting the 50000 , 32*32*3 images into 50000 * 3072 arrays
x_train = x_train.reshape(50000, 3072)
x_test = x_test.reshape(10000, 3072)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.flatten()
y_test = y_test.flatten()

# normalize the datasets
x_train /= 255.
x_test /= 255.

print(x_train.shape , "train shape")
print(x_test.shape , "test shape")
print(y_train.shape , "train shape")
print(y_test.shape , "test shape")

50000 train samples
10000 test samples
(50000, 3072) train shape
(10000, 3072) test shape
(50000,) train shape
(10000,) test shape


In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(600, activation="relu"))
model.add(Dense(400, activation="relu"))
model.add(Dense(200, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="softmax"))  #Last layer with number of classes

In [76]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [77]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=3)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [78]:
y_hat = model.predict(x_test).argmax(axis=1)

313/313 [==============================] - 3s 9ms/step


In [79]:
accuracy_score(y_test, y_hat)

0.4875

In [80]:
pd.crosstab(y_test, y_hat)

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,584,57,75,32,46,16,19,22,116,33
1,46,672,19,33,16,16,15,22,56,105
2,78,32,372,71,148,86,119,47,22,25
3,24,39,100,312,76,211,136,35,26,41
4,49,23,164,63,432,55,121,58,27,8
5,26,33,104,172,79,407,99,46,20,14
6,5,40,93,71,117,61,560,20,14,19
7,51,38,86,71,108,69,39,496,10,32
8,111,106,29,20,36,14,9,16,627,32


In [81]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.56      0.58      0.57      1000
           1       0.51      0.67      0.58      1000
           2       0.35      0.37      0.36      1000
           3       0.35      0.31      0.33      1000
           4       0.40      0.43      0.41      1000
           5       0.43      0.41      0.42      1000
           6       0.49      0.56      0.52      1000
           7       0.61      0.50      0.55      1000
           8       0.64      0.63      0.63      1000
           9       0.57      0.41      0.48      1000

    accuracy                           0.49     10000
   macro avg       0.49      0.49      0.49     10000
weighted avg       0.49      0.49      0.49     10000



In [ ]:
def plot_scores(histo):
    accuracies = histo.history['accuracy']
    val_accuracies = histo.history['val_accuracy']
    epochs = range(len(accuracies))
    plt.plot(epochs, accuracies, label='Accuracy on train')
    plt.plot(epochs, val_accuracies, label='Accuracy on test')
    plt.title('Scores')
    plt.legend()
    plt.show()
plot_scores(history)

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import *
from xgboost import XGBClassifier

In [85]:
model3 = RandomForestClassifier()
model3.fit(x_train,y_train)
y_hat = model3.predict(x_test)
accuracy_score(y_test, y_hat)

0.4685

In [86]:
model1 = LogisticRegression()
model1.fit(x_train,y_train)
y_hat = model1.predict(x_test)
accuracy_score(y_test, y_hat)

0.4051

In [87]:
model2 = XGBClassifier()
model2.fit(x_train,y_train)
y_hat = model2.predict(x_test)
accuracy_score(y_test, y_hat)

KeyboardInterrupt: 

In [49]:
# fit a Xgboost model to the data
model = xgb.XGBClassifier(learning_rate=0.1,
                    n_estimators=10,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0.5,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softmax',
                    nthread=4,
                    num_class=10,
                    seed=27)

In [45]:
# start_time = time.time()
# cv_results = cross_val_score(model, x_train, y_train, cv = 2, scoring='accuracy', n_jobs = 2, verbose = 3)    

In [50]:
start_time = time.time()
model.fit(x_train, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)

print("Execution Time %s seconds: " % (time.time() - start_time)) 

XGBoostError: [13:22:18] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1\xgboost\xgboost-ci-windows\include\xgboost/objective.h:98: multioutput is not supported by current objective function

In [ ]:
# make predictions for test data
y_pred = model.predict(x_test)

predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)


# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()


# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, log_loss
from keras.datasets import cifar10

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Flatten image data into 1D vectors
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Define logistic regression model
logreg = LogisticRegression()

# Define K-fold cross-validation
kf = KFold(n_splits=3, shuffle=True)

# Initialize lists to store training and validation metrics
train_acc = []
val_acc = []
train_loss = []
val_loss = []

# Loop through each fold
for train_idx, val_idx in kf.split(X_train, y_train):
    # Split data into training and validation sets for this fold
    X_train_fold, y_train_fold = X_train[train_idx], y_train[train_idx]
    X_val_fold, y_val_fold = X_train[val_idx], y_train[val_idx]
    
    # Train logistic regression model on this fold
    logreg.fit(X_train_fold, y_train_fold)
    
    # Evaluate training and validation accuracy and loss for this fold
    y_train_pred = logreg.predict(X_train_fold)
    y_val_pred = logreg.predict(X_val_fold)
    train_acc_fold = accuracy_score(y_train_fold, y_train_pred)
    val_acc_fold = accuracy_score(y_val_fold, y_val_pred)
    train_loss_fold = log_loss(y_train_fold, logreg.predict_proba(X_train_fold))
    val_loss_fold = log_loss(y_val_fold, logreg.predict_proba(X_val_fold))
    
    # Append metrics to lists
    train_acc.append(train_acc_fold)
    val_acc.append(val_acc_fold)
    train_loss.append(train_loss_fold)
    val_loss.append(val_loss_fold)

# Plot training and validation accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Logistic Regression Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Fold')
plt.ylabel('Loss')
plt.title('Logistic Regression Training and Validation Loss')
plt.legend()
plt.show()


In [88]:
!pip install git+https://github.com/fmfn/BayesianOptimization

  Cloning https://github.com/fmfn/BayesianOptimization to c:\users\aish0\appdata\local\temp\pip-req-build-2o9z5m1h
  Resolved https://github.com/fmfn/BayesianOptimization to commit 33b99ec0a4fc51239e1a2fca3eaa37ad6debac5d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.4.3-py3-none-any.whl size=18333 sha256=0480930acf5bd2cfe24b70df54dd1ea2166180f2c1c2d9b3844033f2266c5028
  Stored in directory: C:\Users\aish0\AppData\Local\Temp\pip-ephem-wheel-cache-x4xal2_q\wheels\de\4b\fc\ca63ecde7529784254ce89a4c60adafcb973a6b20aa9f18dd4
Successfully built bayesian-optimization
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5


  Running command git clone --filter=blob:none --quiet https://github.com/fmfn/BayesianOptimization 'C:\Users\aish0\AppData\Local\Temp\pip-req-build-2o9z5m1h'
